In [ ]:
%pip install -U spacy

In [2]:
import spacy
import pandas as pd
import openpyxl

In [2]:
!python -m certifi

c:\Users\WS595GW\AppData\Local\Programs\Python\Python38\lib\site-packages\certifi\cacert.pem


In [7]:
%pip install goto-statement

Note: you may need to restart the kernel to use updated packages.


In [8]:
from goto import with_goto

In [4]:
%%python -m spacy download en_core_web_md


UsageError: %%python is a cell magic, but the cell body is empty.


In [ ]:
!pip install https://github.com/MartinoMensio/spacy-universal-sentence-encoder/releases/download/v0.4.5/en_use_md-0.4.5.tar.gz#en_use_md-0.4.5

In [3]:
nlp=spacy.load('en_use_md')

In [4]:
#@title Tuple version-->Use this to create output to CSV
def speaker_split(x):
  turn=[]
  for index,turns in enumerate(x):
    if index%2==0:
      turn.append(turns)
  return turn
  
def speaker_split_2(x):
  turn=[]
  for index,turns in enumerate(x):
    if index%2!=0:
      turn.append(turns)
  return turn  

def parser(keyword_list,sentence_list,conf):
  out_list=[]
  filt_out_li=[]
  for sentence in sentence_list:
    for words in keyword_list:
      out_list.append(transcript_comber_rect_tuple(words,sentence,conf))
  for items in out_list:
    if items!=None:
      filt_out_li.append(items)
  return filt_out_li

def transcript_comber_rect_tuple(phrase,sentence,min_similarity):
  phrases_tokens=nlp(phrase)
  sentence_tokens=nlp(sentence)
  dict={'pharse_checked':phrase,
          'phrase_matched':[],
        'similarity_matched':[]}
  for index,value in enumerate(sentence_tokens):
    if index+len(phrases_tokens)<len(sentence_tokens):
      if len(phrases_tokens)<6:
        if phrases_tokens.similarity(sentence_tokens[index:index+len(phrases_tokens)+1])>min_similarity:
          dict['phrase_matched'].append(sentence_tokens[index:index+len(phrases_tokens)+1])
          dict['similarity_matched'].append(phrases_tokens.similarity(sentence_tokens[index:index+len(phrases_tokens)+1]))
      
      if phrases_tokens.similarity(sentence_tokens[index:index+len(phrases_tokens)])>min_similarity:
        dict['phrase_matched'].append(sentence_tokens[index:index+len(phrases_tokens)])
        dict['similarity_matched'].append(phrases_tokens.similarity(sentence_tokens[index:index+len(phrases_tokens)]))
  
      if len(phrases_tokens)>3:
        if phrases_tokens.similarity(sentence_tokens[index:index+len(phrases_tokens)-1])>min_similarity:
          dict['phrase_matched'].append(sentence_tokens[index:index+len(phrases_tokens)-1])
          dict['similarity_matched'].append(phrases_tokens.similarity(sentence_tokens[index:index+len(phrases_tokens)-1]))
  if len(dict['phrase_matched'])!=0:
    return dict['pharse_checked'],dict['phrase_matched'],dict['similarity_matched'],dict['similarity_matched'].index(max(dict['similarity_matched'])),dict['phrase_matched'][dict['similarity_matched'].index(max(dict['similarity_matched']))]




def analyser(speaker,row,categorie,confi):
    list_turn=ana_df[speaker][row]
    list_keyword=[items for items in key_df[categorie] if len(str(items))>3]
    for items in parser(list_keyword,list_turn,confi):
        print(f"keyword_in rep=>{items[0]}----phrase detected==>{items[4]}")

In [5]:
import numpy as np

In [ ]:
list_keyword=[items for items in key_df['Account Unlock'] if len(str(items))>3]
list_keyword

In [6]:

key_df=pd.read_excel('C:/sim_engine/key_repo_colabo.xlsx',sheet_name='keyphrases')
ana_df=pd.read_csv('C:/sim_engine/comb_out.csv')#input file containing all the collated results
print(ana_df.columns)
print(key_df.columns)
print(type(key_df['Access/Permissions'][0]))

Index(['Unnamed: 0', 'call ids', 'session ids', 'transcript', 'intents'], dtype='object')
Index(['Account Unlock', 'Access/Permissions', 'Password Reset',
       'User Software', 'User Hardware'],
      dtype='object')
<class 'str'>


In [7]:
ana_df['turns_in_list']=ana_df['transcript'].apply(lambda x: x.split('\r\n'))
ana_df['speaker_1']=ana_df['turns_in_list'].apply(speaker_split)
ana_df['speaker_2']=ana_df['turns_in_list'].apply(speaker_split_2)
ana_df['intents_detected']=ana_df['intents'].apply(lambda x: x.split('\n') )
ana_df['intents to check']=["none"]*len(ana_df['call ids'])
for index,items in enumerate(ana_df['intents_detected']):
  to_be_checked=[]
  for item in items:
    if item.lower()!='no flows detected yet' and len(item)>3 and not item.lower().__contains__('access'):
      to_be_checked.append(item.replace('\r',''))
    elif item.lower().__contains__('access'):
      to_be_checked.append('Access/Permissions')
    else:
      to_be_checked.append("none")
  print(f"{ana_df['call ids'][index]}-------->{to_be_checked}")
  ana_df['intents to check'][index]=to_be_checked
ana_df.to_csv(r'C:/sim_engine/output_for_ref.csv', header=True, index=True)

User_Software_293243240077784_188814_8145051_07-11-2022-15-41-00.wav-------->['Access/Permissions', 'User Hardware', 'User Software']
User_Software_293243240078546_143034_8146941_07-11-2022-17-14-00.wav-------->['none']
User_Software_293243323710382_187998_8146108_07-11-2022-16-20-00.wav-------->['User Software']
User_Software_293247535079548_214121_8242416_07-15-2022-10-42-00.wav-------->['none']
User_Software_293247535082811_122057_8255450_07-15-2022-15-11-00.wav-------->['User Hardware', 'Access/Permissions', 'User Software']
User_Software_293247535083375_120562_8258679_07-15-2022-16-36-00.wav-------->['User Software']
User_Software_293247535101274_187880_8307355_07-19-2022-08-17-00.wav-------->['none']
User_Software_293247618704128_123039_8228125_07-14-2022-15-52-00.wav-------->['none']
User_Software_293247618704415_120562_8229701_07-14-2022-16-33-00.wav-------->['Access/Permissions']
User_Software_293247618707960_180284_8240317_07-15-2022-10-02-00.wav-------->['User Hardware']
Use

C:\Users\WS595GW\AppData\Local\Temp\ipykernel_15660\2856015615.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ana_df['intents to check'][index]=to_be_checked


In [29]:
list_turns=ana_df['speaker_2'][6]
list_keywords=key_df['User Software'][:30]
for items in parser(list_keywords,list_turns,0.6):
  print(f"keyword_in rep=>{items[0]}----phrase detected==>{items[4]}")

keyword_in rep=>power chart is not opening----phrase detected==>power chart in cerner
keyword_in rep=>not allowing me to click to open the power chart----phrase detected==>i was supposed to get access for power chart in


Enter the details to start checking for individual(not recoomended)

In [7]:
turns_in_channel='speaker_2'
checker=True
counter=0
stop=0
swap=False
loop_term=False
while checker is True:
    category_selection=int(input("enter the category to be selected: \n 1- Account Unlock 2- Password Reset \n 3- Access/perm 4- User Software 5-User Hardware \n 6-stop process!!!! \n "))
    row_selection=int(input("enter the row from the output_fin sheet : "))
    change_speaker_channel=input("change speaker 1 and 2,(default is 2): y/n: ")
    if change_speaker_channel.lower()=='y':
        if turns_in_channel.__contains__('_2'):
            turns_in_channel='speaker_1'
            swap=True
        else:
            pass
    if category_selection==1 and stop!=1:
        print('Account Unlock selected ')
        analyser(turns_in_channel,row_selection,'Account Unlock',0.6)
    elif category_selection==2 and stop!=1:
        print('Password Reset selected ')
        analyser(turns_in_channel,row_selection,'Password Reset',0.6)
    elif category_selection==3 and stop!=1:
        print('Access/Permissions selected ')
        analyser(turns_in_channel,row_selection,'Access/Permissions',0.6)
    elif category_selection==4 and stop!=1:
        print('User Software selected ')
        analyser(turns_in_channel,row_selection,'User Software',0.6)
    elif category_selection==5 and stop!=1:
        print('User Hardware selected ')
        analyser(turns_in_channel,row_selection,'User Hardware',0.6)
    elif category_selection==6 or stop==1:
        print("termintating : bye bye")
        checker = False
    else:
        print("enter valid selection")
        checker = False
    turns_in_channel='speaker_2'
    if swap==True:
        print('swapping to default channel:speaker_2')
        swap=False
    stop=int(input("stop?(1/0): "))
    print('\n')

termintating : bye bye




ALL Checker

In [41]:



def all_checker(turns_in_channel,row_select,loop_term):
    row_selection=row_select
    checker_loop=loop_term
    while checker_loop!=True:
        confide=float(input("please enter a confidence value: reccomended bw 0.75 and 0.9: "))
        change_speaker_channel=input("change speaker 1 and 2,(default is 2): y/n: ")
        if change_speaker_channel.lower()=='y':
            if turns_in_channel.__contains__('_2'):
                turns_in_channel='speaker_1'
                swap=True
            else:
                pass
        print('\nacc unlock detected:')
        analyser(turns_in_channel,row_selection,'Account Unlock',confide)
        print('\nPassword Reset deteceted:')
        analyser(turns_in_channel,row_selection,'Password Reset',confide)
        print('\nAccess/Permissions:')
        analyser(turns_in_channel,row_selection,'Access/Permissions',confide)
        print('\nUser Software:')
        analyser(turns_in_channel,row_selection,'User Software',confide)
        print('\nUser Hardware:')
        analyser(turns_in_channel,row_selection,'User Hardware',confide)
        turns_in_channel='speaker_2'
        change_row=input("change rows to analyze? y/n: ")
        if change_row.lower()=='y':
            row_selection=int(input("enter the row from the output_fin sheet : "))
        else:
            pass

        stop_all_check=int(input("stop?(1/0): "))
        if stop_all_check==1:
            checker_loop=True
        else:
            pass




In [ ]:
turn_in_channel='speaker_2'
checker=True
counter=0
stop=0
swap=False
loop_term=False

rows_selection=int(input("enter the row from the output_fin sheet : "))
print(f"row selected {row_selection}\n")
all_checker(turn_in_channel,rows_selection,loop_term)


UI version

In [8]:
def analyzer_ui(speake_select,row_num,issue_type,confidence):
    out_re=[]
    keyphrases_detected=[]
    list_turns=ana_df[f"speaker_{speake_select}"][row_num]
    list_keywords=[items for items in key_df[issue_type] if len(str(items))>3]
    for items in parser(list_keywords,list_turns,confidence):
        out_re.append(f"keyword_in rep=>{items[0]}----phrase detected==>{items[4]}")
        keyphrases_detected.append(items[4])
    return out_re,keyphrases_detected

def all_checker_ui(turn_in_chanl,row_select,confide,check_list):
    to_be_highlighted={}
    check_li=check_list.split(",")
    ref_dict={'1':'Account Unlock','2':'Password Reset','5':'User Hardware','4':'User Software','3':'Access/Permissions'}
    for item in check_li:
        text_box_out.insert('end',f'\n {ref_dict[item]} detected:\n----------------------------\n ')
        item_list,keyphrase_list=analyzer_ui(turn_in_chanl,row_select,ref_dict[item],confide)
        for items in item_list:
            text_box_out.insert('end',f"{items}\n\n")
        to_be_highlighted[item]=keyphrase_list
    return to_be_highlighted
    



In [106]:
li='2'
li_list=li.split(",")
print(li_list)

['2']


In [9]:
import tkinter as tk
from tkinter import font


root = tk.Tk()
my_font = font.Font(weight='bold')
root.geometry("1000x600")
root.config(bd=10, relief="sunken",bg='#482c40')
for i in range(10):
    root.grid_columnconfigure(i, weight=1, minsize=10)
    root.grid_rowconfigure(i, weight=1, minsize=10)

label1 = tk.Label(root, text="enter the row number(refer the out):         ")
label1.grid(row=0, column=0,sticky="nsew")

entry1 = tk.Entry(root)
entry1.grid(row=0, column=1)

label2 = tk.Label(root, text="enter the category to be selected: \n 1- Account Unlock   2- Password Reset   3- Access/perm\n 4- User Software   5-User Hardware \n  ")
label2.grid(row=0, column=2,padx=10,pady=5,sticky="nsew")

entry2 = tk.Entry(root)
entry2.grid(row=0, column=3)

label3 = tk.Label(root, text="enter the speaker channel(1 or 2)")
label3.grid(row=2, column=0,padx=10,sticky="nsew")

entry3 = tk.Entry(root)
entry3.grid(row=2, column=1,padx=10,sticky="nsew")

label4 = tk.Label(root, text="enter the confidence level(0.2-0.99): ")
label4.grid(row=1, column=2,padx=10)

entry4 = tk.Entry(root)
entry4.grid(row=1, column=3,padx=10,sticky="nsew")


label5 = tk.Label(root, text="call id: \nsession id: ")
label5.grid(row=3, column=0,pady=10,)
label6 = tk.Label(root, text="detected by collabo: ")
label6.grid(row=3, column=2,pady=10,)
text_box_transcript = tk.Text(root, height=28, width=50)
text_box_transcript.grid(row=4, column=0, rowspan=10 ,columnspan=2, pady=10,padx=5,sticky="nsew")


text_box_out = tk.Text(root, height=28, width=80)
text_box_out.grid(row=4, column=2, rowspan=10 ,columnspan=4, pady=10,padx=5,sticky="nsew")


def show_values():
    text_box_transcript.delete("1.0", tk.END)
    text_box_out.delete("1.0", tk.END)
    row_number = int(entry1.get())
    selection_op = entry2.get()
    print(selection_op)
    speaker_selection=int(entry3.get())
    confi_entry=float(entry4.get())
    label5.config(text=f"call id:{ana_df['call ids'][row_number]} \nsession id:{ana_df['session ids'][row_number]}")
    label6.config(text=f"AI detected: {ana_df['intents to check'][row_number]}")
    if speaker_selection==1 or speaker_selection==2:
        
        for items in ana_df[f'speaker_{speaker_selection}'][row_number]:
             text_box_transcript.insert(tk.END,f"{items}\n\n")
        list_highgiht=all_checker_ui(speaker_selection,row_number,float(confi_entry),selection_op)
        print(list_highgiht)
    else:
        text_box_transcript.insert(tk.END,"please enter a value of 1 and 2")
    
    

confirm_button = tk.Button(root, text="Confirm", command=show_values)
confirm_button.grid(row=0, column=4,pady=10,sticky="nsew")

root.mainloop()


Hola modified UI version

In [12]:
import tkinter as tk
from tkinter import font
import random

root = tk.Tk()
my_font = font.Font(weight='bold',size='12')
root.geometry("1100x600")
root.config(bd=10, relief="sunken",bg='#482c40')
for i in range(10):
    root.grid_columnconfigure(i, weight=1, minsize=10)
    root.grid_rowconfigure(i, weight=1, minsize=10)

label1 = tk.Label(root, text="enter the row number(refer the out):         ")
label1.grid(row=0, column=0,sticky="nsew")

entry1 = tk.Entry(root)
entry1.grid(row=0, column=1)

label2 = tk.Label(root, text="enter the category to be selected: \n 1- Account Unlock   2- Password Reset   3- Access/perm\n 4- User Software   5-User Hardware \n  ")
label2.grid(row=0, column=2,padx=10,pady=5,sticky="nsew")

entry2 = tk.Entry(root)
entry2.grid(row=0, column=3)

label3 = tk.Label(root, text="enter the speaker channel(1 or 2)")
label3.grid(row=2, column=0,padx=10,sticky="nsew")

entry3 = tk.Entry(root)
entry3.grid(row=2, column=1,padx=10,sticky="nsew")

label4 = tk.Label(root, text="enter the confidence level(0.2-0.99): ")
label4.grid(row=1, column=2,padx=10)

entry4 = tk.Entry(root)
entry4.grid(row=1, column=3,padx=10,sticky="nsew")


label5 = tk.Label(root, text="call id: \nsession id: ")
label5.grid(row=3, column=0,pady=10,)
label6 = tk.Label(root, text="detected by collabo: ")
label6.grid(row=3, column=2,pady=10,)
text_box_transcript = tk.Text(root, height=28, width=50)
text_box_transcript.grid(row=4, column=0, rowspan=10 ,columnspan=2, pady=10,padx=5,sticky="nsew")


text_box_out = tk.Text(root, height=28, width=80)
text_box_out.grid(row=4, column=2, rowspan=10 ,columnspan=4, pady=10,padx=5,sticky="nsew")


def show_values():
    color_li=['yellow','red','blue','green','orange']
    text_box_transcript.delete("1.0", 'end')
    text_box_out.delete("1.0", 'end')
    row_number = int(entry1.get())
    selection_op = entry2.get()
    print(selection_op)
    speaker_selection=int(entry3.get())
    confi_entry=float(entry4.get())
    label5.config(text=f"call id:{ana_df['call ids'][row_number]} \nsession id:{ana_df['session ids'][row_number]}")
    label6.config(text=f"AI detected: {ana_df['intents to check'][row_number]}")
    
  
    if speaker_selection==1 or speaker_selection==2:
        
        for items in ana_df[f'speaker_{speaker_selection}'][row_number]:
             text_box_transcript.insert('end',f"{items}\n\n")
        dict_highgiht=all_checker_ui(speaker_selection,row_number,float(confi_entry),selection_op)
        #print(dict_highgiht)
        for key in dict_highgiht:
            selected_color=random.choice(color_li)
            list_highlight_refined=[]
            for items in dict_highgiht[key]:
                if str(items) not in list_highlight_refined:
                    list_highlight_refined.append(str(items))
            for word in list_highlight_refined:
                start = '1.0'
                while True:
                    start = text_box_transcript.search(word, start, stopindex='end')
                    if not start:
                        break
                    end = f"{start} + {len(word)}c"
                    text_box_transcript.tag_add(f"highlight_{key}", start, end)
                    start = end
            text_box_transcript.tag_config(f"highlight_{key}", background=selected_color,font=my_font)
            color_li.remove(selected_color)
            selected_color=random.choice(color_li)
            #print(color_li)
    else:
        text_box_transcript.insert('end',"please enter a value of 1 and 2")
    
    

confirm_button = tk.Button(root, text="Confirm", command=show_values)
confirm_button.grid(row=0, column=4,pady=10,sticky="nsew")

root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\WS595GW\AppData\Local\Programs\Python\Python38\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "C:\Users\WS595GW\AppData\Local\Temp\ipykernel_15660\2270681177.py", line 67, in show_values
    dict_highgiht=all_checker_ui(speaker_selection,row_number,float(confi_entry),selection_op)
  File "C:\Users\WS595GW\AppData\Local\Temp\ipykernel_15660\2150636108.py", line 16, in all_checker_ui
    text_box_out.insert('end',f'\n {ref_dict[item]} detected:\n----------------------------\n ')
KeyError: ''


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\WS595GW\AppData\Local\Programs\Python\Python38\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "C:\Users\WS595GW\AppData\Local\Temp\ipykernel_15660\2270681177.py", line 67, in show_values
    dict_highgiht=all_checker_ui(speaker_selection,row_number,float(confi_entry),selection_op)
  File "C:\Users\WS595GW\AppData\Local\Temp\ipykernel_15660\2150636108.py", line 16, in all_checker_ui
    text_box_out.insert('end',f'\n {ref_dict[item]} detected:\n----------------------------\n ')
KeyError: ''


3,4,5
3,4,5
5
5
5
5
5
5
5
5
5
5
5
3
3
3
4
3
3
3
3
3
3
4



Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\WS595GW\AppData\Local\Programs\Python\Python38\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "C:\Users\WS595GW\AppData\Local\Temp\ipykernel_15660\2270681177.py", line 67, in show_values
    dict_highgiht=all_checker_ui(speaker_selection,row_number,float(confi_entry),selection_op)
  File "C:\Users\WS595GW\AppData\Local\Temp\ipykernel_15660\2150636108.py", line 16, in all_checker_ui
    text_box_out.insert('end',f'\n {ref_dict[item]} detected:\n----------------------------\n ')
KeyError: ''


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\WS595GW\AppData\Local\Programs\Python\Python38\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "C:\Users\WS595GW\AppData\Local\Temp\ipykernel_15660\2270681177.py", line 67, in show_values
    dict_highgiht=all_checker_ui(speaker_selection,row_number,float(confi_entry),selection_op)
  File "C:\Users\WS595GW\AppData\Local\Temp\ipykernel_15660\2150636108.py", line 16, in all_checker_ui
    text_box_out.insert('end',f'\n {ref_dict[item]} detected:\n----------------------------\n ')
KeyError: ''


...

4
5



Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\WS595GW\AppData\Local\Programs\Python\Python38\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "C:\Users\WS595GW\AppData\Local\Temp\ipykernel_15660\2270681177.py", line 67, in show_values
    dict_highgiht=all_checker_ui(speaker_selection,row_number,float(confi_entry),selection_op)
  File "C:\Users\WS595GW\AppData\Local\Temp\ipykernel_15660\2150636108.py", line 16, in all_checker_ui
    text_box_out.insert('end',f'\n {ref_dict[item]} detected:\n----------------------------\n ')
KeyError: ''


3
4
4
4
4
4
4
4
4
4
1



Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\WS595GW\AppData\Local\Programs\Python\Python38\lib\tkinter\__init__.py", line 1883, in __call__
    return self.func(*args)
  File "C:\Users\WS595GW\AppData\Local\Temp\ipykernel_15660\2270681177.py", line 67, in show_values
    dict_highgiht=all_checker_ui(speaker_selection,row_number,float(confi_entry),selection_op)
  File "C:\Users\WS595GW\AppData\Local\Temp\ipykernel_15660\2150636108.py", line 16, in all_checker_ui
    text_box_out.insert('end',f'\n {ref_dict[item]} detected:\n----------------------------\n ')
KeyError: ''


3
3
1
1
5
5
5


In [27]:
dict_a={1:12,2:22,3:33}
for key in dict_a:
    print(dict_a[key])

12
22
33
